In [ ]:
## Instalamos la libreria psycopg2

%pip install psycopg2

In [3]:
## Librerias a importar

import requests
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values

In [4]:
## Conexion a API

url = 'https://rickandmortyapi.com/api/character'
headers = {"Accept-Encoding": "gzip, deflate"}

response = requests.get(url, headers = headers)
data = response.json()

df = pd.json_normalize(data['results'])
df = df.drop(['episode'], axis=1)

In [ ]:
## Creacion de tabla en codigo SQL para Redshift

""" CREATE TABLE IF NOT EXISTS federicohernansanches_coderhouse.characters (
	id INT,
	name VARCHAR(255),
	status VARCHAR(255),
	species VARCHAR(255),
	type VARCHAR(255),
	gender VARCHAR(255),
	image VARCHAR(255),
	url VARCHAR(255),
	origin_name VARCHAR(255),
	origin_url VARCHAR(255),
	location_name VARCHAR(255),
	location_url VARCHAR(255),
	created	TIMESTAMPTZ
) DISTKEY(id) SORTKEY(name) """

In [ ]:
## Creo string para insertar y paso valores a lista

cols = ['id', 'name', 'status', 'species', 'type', 'gender', 'image', 'url', 'created', 'origin_name', 'origin_url', 'location_name', 'location_url']

insert_sql = f"INSERT INTO federicohernansanches_coderhouse.characters ({', '.join(cols)}) VALUES %s"

df2 = df.head(5) ## Agarro las primeras 5 a modo de ejemplo para no sobrecargar la base; en el TP final cargare todo
values = [tuple(x) for x in df2.to_numpy()]

values

In [5]:
## Conexion a DB Redshift

url = "data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
data_base = "data-engineer-database"
user = "federicohernansanches_coderhouse"
pwd = "6H0Tll7uih"

try:
    conn = psycopg2.connect(
        host = url,
        dbname = data_base,
        user = user,
        password = pwd,
        port = '5439'
    )
    print("Connected to Redshift successfully!")
    
except Exception as e:
    print("Unable to connect to Redshift.")
    print(e)

Connected to Redshift successfully!


In [7]:
## Cargo datos en la tabla de redshift

cur = conn.cursor()

cur.execute("BEGIN")
execute_values(cur, insert_sql, values)
cur.execute("COMMIT")

print('Proceso terminado')

conn.close()

Proceso terminado
